### First Neccesary libararies needs to be loaded.

In [19]:
import gradio as gr
import transformers
from transformers import pipeline
import tf_keras as keras
import pandas as pd

### Now we import our already pre-trained model from 

In [2]:
# Load pre-trained spam classifier
spam_classifier = pipeline(
    "text-classification",
    model="mrm8488/bert-tiny-finetuned-sms-spam-detection"
)

model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/324 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use 0


In [16]:
import tempfile

### Since single email check is hassle we will make a function for batch classication
### we should assume certain file template or format so our program knows what to expect

In [32]:
def classify_batch(file):
    """Process uploaded CSV/TXT file with multiple emails"""
    results = []
    if file.name.endswith('.csv'):    # Handling the emails in CSV files format
        df = pd.read_csv(file)
        emails = df['email'].tolist()  # we assume there's a column named 'email'
        for idx, email in enumerate(emails):
            prediction = spam_classifier(email)[0]
            results.append({
                "email": email[:50] + "...",  # Truncate for display
                "label": "SPAM" if prediction["label"] == "LABEL_1" else "HAM",
                "confidence": prediction["score"]
            })

    ### Now we almost do the same thing but for text files (one email per line)
    elif file.name.endswith('.txt'):
        with open(file.name, 'r') as f:
            emails = f.readlines()
            for email in emails:
                prediction = spam_classifier(email.strip())[0]
                results.append({
                    "email": email.strip()[:50] + "...",
                    "label": "SPAM" if prediction["label"] == "LABEL_1" else "HAM",
                    "confidence": f"{prediction['score']:.4f}"
                })
        ### --------------- Here we implemnt some condition for our uploaded files __________
        try:
            results = []
            if not file.name:
                raise gr.Error("No file uploaded")
                # Handle CSV files
            if file.name.endswith('.csv'):
                df = pd.read_csv(file)
                if 'email' not in df.columns:
                    raise gr.Error("CSV file must contain 'email' column")
                emails = df['email'].tolist()
            
        # Handle text files
            elif file.name.endswith('.txt'):
                with open(file.name, 'r') as f:
                    emails = f.readlines()
            else:
                raise gr.Error("Unsupported file format. Only CSV/TXT accepted")
                
            # Limit to 100 emails max for demo
            emails = emails[:100]


        except gr.Error as e:
            raise e  # Re-raise Gradio errors to show pop-up
        except Exception as e:
            raise gr.Error(f"An unexpected error occurred: {str(e)}")


    return pd.DataFrame(results)

In [35]:
def classify_batch(file):
    """Process uploaded CSV/TXT file with multiple emails"""
    try:
        results = []
        
        # Check if file exists
        if not file.name:
            raise gr.Error("No file uploaded")

        # --- CSV File Handling ---
        if file.name.endswith('.csv'):
            df = pd.read_csv(file)
            
            # Check for required email column
            if 'email' not in df.columns:
                raise gr.Error("CSV file must contain a column named 'email'")
                
            emails = df['email'].tolist()

        # --- Text File Handling ---
        elif file.name.endswith('.txt'):
            with open(file.name, 'r') as f:
                emails = f.readlines()
        
        # --- Unsupported Format ---
        else:
            raise gr.Error("Unsupported file format. Only CSV/TXT accepted")

        # Process emails (common for both formats)
        emails = emails[:100]  # Limit to 100 emails
        for email in emails:
            # Handle empty lines in text files
            if not email.strip():
                continue
                
            prediction = spam_classifier(email.strip())[0]
            results.append({
                "email": email.strip()[:50] + "...",
                "label": "SPAM" if prediction["label"] == "LABEL_1" else "HAM",
                "confidence": f"{prediction['score']:.4f}"
            })

        return pd.DataFrame(results)

    except gr.Error as e:
        raise e  # Show pop-up for expected errors
    except Exception as e:
        raise gr.Error(f"Processing error: {str(e)}")

### We define simple function for classification

In [36]:
def classify_text(text):
    result = spam_classifier(text)[0]
    return {
        "Spam": result["score"] if result["label"] == "LABEL_1" else 1 - result["score"],
        "Ham": result["score"] if result["label"] == "LABEL_0" else 1 - result["score"]
    }

### using gradio we will make a simple interface for our program

In [21]:
import pandas as pd

# Data
emails = [
    "Hi John, just wanted to follow up about tomorrow's meeting. Please confirm your attendance.",
    "Congratulations! You've won a free cruise vacation! Click here to claim your prize >>>",
    "Your Amazon order #456789 has shipped. Track your package: https://amzn.track/456789",
    "URGENT: Your bank account requires immediate verification. Login now to avoid suspension.",
    "Reminder: Team lunch today at 1 PM in the main conference room."
]

# Create DataFrame
df = pd.DataFrame(emails, columns=["email"])

# Save to CSV
df.to_csv("sample_emails.csv", index=False)

In [54]:
with gr.Blocks(title="Spam Classifier Pro") as demo:
    gr.Markdown("# 📧 Welcome to Spamedar!")
    
    
    with gr.Tab("✉️ Single Email"):
        gr.Interface(
            description.Markedown="Copy your email to find out if it's a is Spam or Ham.👇",
            fn=classify_text,
            inputs=gr.Textbox(label="Input Email", lines=3),
            outputs=gr.Label(label="Classification"),
            examples=[
                ["Urgent: Verify your account details now!"],
                ["Hey, can we meet tomorrow to discuss the project?"],
                ["WINNER! You've been selected for a $1000 Walmart Gift Card!"],
                ["Your account needs verification. Click here to confirm your details."],
                ["Meeting rescheduled to Friday 2 PM"]
            ]
        )
    
    with gr.Tab("📨 Multiple Emails"):
        gr.Markdown("## Upload email batch (CSV or TXT)")
        file_input = gr.File(label="Upload File", file_types=[".csv", ".txt"])
        output_table = gr.Dataframe(
            headers=["email", "label", "confidence"],
            datatype=["str", "str", "number"],
            interactive=False,
            label="Classification Results"
        )
        download_btn = gr.DownloadButton(label="Download Results")

        def process_file(file):
            """Process file and return (display_df, download_path)"""
            results_df = classify_batch(file)
            
            with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as f:
                results_df.to_csv(f.name, index=False)
                return results_df, f.name
        
        file_input.upload(
            fn=process_file,
            inputs=file_input,
            outputs=[output_table, download_btn]  # Update both components
    )
        
        gr.Examples(
            examples=[
                ["sample_emails.csv"],
                ["batch_emails.txt"]
            ],
            inputs=file_input
        )
if __name__ == "__main__":
    demo.launch()

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (3782653024.py, line 7)

In [17]:
demo = gr.Interface(
    fn=classify_text,
    inputs=gr.Textbox(label="Email/Message", placeholder="Enter text here..."),
    outputs=gr.Label(label="Classification Results"),
    title="Spamedar",
    description="Copy your email to find out if it's a is Spam or Ham.👇",
    examples=[
        ["Hey, can we meet tomorrow to discuss the project?"],
        ["WINNER! You've been selected for a $1000 Walmart Gift Card!"],
        ["Your account needs verification. Click here to confirm your details."]
    ]
)

demo.launch()

FileNotFoundError: [Errno 2] No such file or directory: 'sample_emails.csv'